In [ ]:
!pip install datasets pandas google-genai numpy tiktoken

  Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
  Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
     ---------------------------------------- 0.0/98.2 kB ? eta -:--:--
     ---- ----------------------------------- 10.2/98.2 kB ? eta -:--:--
     ---------------------------------------- 98.2/98.2 kB 1.4 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [37]:
from datasets import load_dataset
import pandas as pd
import tiktoken

ds = load_dataset("OpenPipe/hacker-news") # streaming=True

In [5]:
ds['train'][-4000]

{'id': 41809385,
 'type': 'comment',
 'by': 'klabb3',
 'time': datetime.datetime(2024, 10, 11, 13, 37, 56),
 'title': None,
 'text': '&gt; Expecting users who type in email addresses to respect case-sensitivity is wishful thinking at best.<p>I agree. First, you have tons of websites using the wrong input field (“text” instead of “email”) which often results in capitalized inputs without user intent. Then you have the non-techies who would absolutely not remember this little gotcha, and put in randomly interchangeable casing depending on who knows what. Some people still thinks capitalization looks more formal and correct, for instance.<p>So what’s the benefit of adhering to the standard strictly? Nothing that solves real-world issues afaik. There is only downside: very simple impersonation attacks.<p>That said, there is a middle ground. Someone put it like this: store and send user input the way they entered it but used the canonical address for testing equality, eg in the database.',


In [54]:
df = pd.DataFrame(ds['train'][-200_000:-40000])

In [26]:
df.tail(3)

,id,type,by,time,title,text,url,score,parent,top_level_parent,descendants,kids,deleted,dead
39996,41813381,comment,marcosdumay,2024-10-11 20:26:21,None,Add forced sedentarism into that set.<p>This i...,None,NaN,41812891.0,41811263,NaN,None,None,None
39997,41813382,story,turkeynecks,2024-10-11 20:26:24,Mini DOOM-like FPS in BooBoo programming language,None,https://www.indiedb.com/games/doomed1,1.0,NaN,41813382,0.0,None,None,None
39998,41813383,comment,davio,2024-10-11 20:26:28,None,hims sells the generic version for a fraction ...,None,NaN,41813102.0,41811263,NaN,None,None,None


In [23]:
def find_ancestors(df, row_id) -> list[int]:
    entry = df[df['id'] == int(row_id)]

    if len(entry) == 0:
        return []
    elif len(entry) > 1:
        raise ValueError(f"Multiple rows with id {row_id}.")
    entry = entry.iloc[0]
    if entry.parent > 0:
        return [entry.id] + find_ancestors(df, entry.parent)
    else:
        return [entry.id]

find_ancestors(df,41813270)

[np.int64(41813270), np.int64(41812813)]

In [31]:
df[df['id'] == find_ancestors(df,41813270)[1]]

,id,type,by,time,title,text,url,score,parent,top_level_parent,descendants,kids,deleted,dead
39428,41812813,story,josephcsible,2024-10-11 19:37:32,"Valve says Steam users don't own a thing, GOG ...",None,https://www.gamesradar.com/games/valve-reminds...,43.0,NaN,41812813,14.0,"[41813270, 41812821, 41813267, 41813289, 41813...",None,None


In [55]:
stories = df[df.type == 'story']
comments = df[df.type == 'comment']

In [59]:
comments

,id,type,by,time,title,text,url,score,parent,top_level_parent,descendants,kids,deleted,dead,tokens
1,41613386,comment,fragmede,2024-09-21 23:22:37,None,Surely you&#x27;re not implying that auto manu...,None,NaN,41613362.0,41612254,NaN,None,None,None,Surely you&#x27;re not implying that auto manu...
2,41613387,comment,jart,2024-09-21 23:22:38,None,It&#x27;s easier on people emotionally to blam...,None,NaN,41612953.0,41612254,NaN,"[41613499, 41613443]",None,None,It&#x27;s easier on people emotionally to blam...
3,41613388,comment,None,2024-09-21 23:22:42,None,None,None,NaN,41613328.0,41612254,NaN,None,True,None,None
4,41613389,comment,Gud,2024-09-21 23:22:53,None,"These are basic items, much of it safety gear....",None,NaN,41606577.0,41606530,NaN,None,None,None,"These are basic items, much of it safety gear...."
5,41613390,comment,nradov,2024-09-21 23:23:07,None,Yikes. I can&#x27;t imagine the circumstances ...,None,NaN,41613219.0,41612578,NaN,[41613436],None,None,Yikes. I can&#x27;t imagine the circumstances ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159995,41773380,comment,nvlled,2024-10-08 02:47:39,None,"Not disagreeing with your overall point, but J...",None,NaN,41771812.0,41766515,NaN,None,None,None,"Not disagreeing with your overall point, but J..."
159996,41773381,comment,debo_,2024-10-08 02:47:59,None,"Yes, I also played a ton of F117A. The terribl...",None,NaN,41773368.0,41769971,NaN,None,None,None,"Yes, I also played a ton of F117A. The terribl..."
159997,41773382,comment,protimewaster,2024-10-08 02:48:00,None,I was thinking more just use a third party ser...,None,NaN,41772688.0,41769657,NaN,None,None,None,I was thinking more just use a third party ser...
159998,41773383,comment,nighthawk454,2024-10-08 02:48:04,None,"Iiuc I think it does work, but the only guaran...",None,NaN,41773343.0,41755719,NaN,[41773401],None,None,"Iiuc I think it does work, but the only guaran..."


## Clean comments

In [68]:
bad_comment = ['[flagged]', '[dead]', 'Thanks!', 'Thank you!', 'Yes.', 'No.', 'Yes', 'No', 'Thanks', 'Thank you']
comments['low_quality'] = comments.tokens.map(lambda x: x in bad_comment) | comments.text.isna()

C:\Users\David\AppData\Local\Temp\ipykernel_20372\447365482.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comments['low_quality'] = comments.tokens.map(lambda x: x in bad_comment) | comments.text.isna()


In [72]:
tokenizer = tiktoken.encoding_for_model('gpt-4o')
comments['tokens'] = comments[~comments.low_quality].text.map(lambda x: len(tokenizer.encode(x)))
comments.tokens.describe()

C:\Users\David\AppData\Local\Temp\ipykernel_20372\2108032067.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comments['tokens'] = comments[~comments.low_quality].text.map(lambda x: len(tokenizer.encode(x)))


count    134468.000000
mean         92.830584
std         113.663063
min           1.000000
25%          27.000000
50%          59.000000
75%         117.000000
max        4679.000000
Name: tokens, dtype: float64

In [ ]:
# drop common comments


np.int64(321)